In [ ]:
# timeit

# Student Name : Sophie Briques
# Cohort       : Castro - 3

################################################################################
# Import Packages
################################################################################

import pandas  as pd   # data science essentials
import numpy   as np   
from   sklearn.model_selection import train_test_split    # train test split
from   sklearn.preprocessing   import StandardScaler      # standard scaler
from   sklearn.linear_model    import LinearRegression    # linear regression (scikit-learn)




################################################################################
# Load Data
################################################################################

# specifying file name
file = "Apprentice_Chef_Dataset.xlsx"

# reading the file into Python
original_df = pd.read_excel(file)
chef_org = original_df.copy()

################################################################################
# Feature Engineering and (optional) Dataset Standardization
################################################################################

#################################################
##########  User-Defined Functions    ###########
#################################################

# Defining function to flag high outliers in variables
def outlier_flag_hi(variable, threshold, data):
    """
    This function is used to flag high outliers in a dataframe the variables' 
    outliers by creating a new column that is preceded by 'out_'.

    PARAMETERS
    ----------
    variable  : str, continuous variable.
    threshold : float, value that will identify where outliers would be.
    data      : dataframe, where the variables are located.
    
    """
    # creating a new column
    data['out_' + variable] = 0
        
    # defining outlier condition
    high = data.loc[0:,'out_' + variable][data[variable] > threshold]
        
    # imputing 1 inside flag column
    data['out_' + variable].replace(to_replace = high,
                                    value   = 1,
                                    inplace = True)

# Defining function to flag high outliers in variables
def outlier_flag_lo(variable, threshold, data):
    """
    This function is used to flag low outliers in a dataframe the variables' 
    outliers by creating a new column that is preceded by 'out_'.

    PARAMETERS
    ----------
    variable  : str, continuous variable.
    threshold : float, value that will identify where outliers would be.
    data      : dataframe, where the variables are located.
    
    """
    # creating a new column
    data['out_' + variable] = 0
        
    # defining outlier condition
    low = data.loc[0:,'out_' + variable][data[variable] < threshold]
        
    # imputing 1 inside flag column
    data['out_' + variable].replace(to_replace = low,
                                    value   = 1,
                                    inplace = True)

    
# Defining function to flag higher variables
def higher_change_flag(variable, index, threshold, data):
    """
    This function is used to flag in a dataframe the variables' trend changes
    above a threshold by creating a new column that is preceded by 'change_'.

    PARAMETERS
    ----------
    variable  : str, continuous variable.
    threshold : float, value that will identify after which the trend on
                variable y changes
    data      : dataframe, where the variables are located.
    
    """
    new_column = 'change_' + variable + "_" + str(index)
    
    # creating a new column
    data[new_column] = 0
        
    # defining outlier condition
    high = data.loc[0:,new_column][data[variable] > threshold]
        
    # imputing 1 inside flag column
    data[new_column].replace(to_replace = high,
                                       value   = 1,
                                       inplace = True)
    
    
# Defining function to flag change at variables
def at_flag(variable, index, threshold, data):
    """
    This function is used to flag in a dataframe the variables' trend changes
    at a threshold by creating a new column that is preceded by 'change_'.

    PARAMETERS
    ----------
    variable  : str, continuous variable.
    threshold : float, value that will identify after which the trend on 
                variable y changes
    data      : dataframe, where the variables are located.
    
    """
    new_column = 'change_' + variable + "_" + str(index)
    
    # creating a new column
    data[new_column] = 0
        
    # defining outlier condition
    high = data.loc[0:,new_column][data[variable] == threshold]
        
    # imputing 1 inside flag column
    data[new_column].replace(to_replace = high,
                                       value   = 1,
                                       inplace = True)

#Defining a function to standardize numerical variables in the dataset:
def standard(num_df):
    """
    This function standardizes a dataframe that contains variables which are 
    either integers or floats.
    
    ------
    num_df : DataFrame, must contain only numerical variables
    
    """
    # Isolating target variable from DF to be standardized
    #num_df = num_df.drop(target_variable, axis = 1)
    
    # INSTANTIATING a StandardScaler() object
    scaler = StandardScaler()

    # FITTING the scaler with housing_data
    scaler.fit(num_df)

    # TRANSFORMING our data after fit
    X_scaled = scaler.transform(num_df)


    # converting scaled data into a DataFrame
    X_scaled_df = pd.DataFrame(X_scaled)
    
    # adding labels to the scaled DataFrame
    X_scaled_df.columns = num_df.columns
    
    # Re-attaching target variable to DataFrame
    #X_scaled_df = X_scaled_df.join(target_variable)
    
    # returning the standardized data frame into the global environment
    return X_scaled_df


#################################################
#############  Feature Engineering    ###########
#################################################

# Flagging missing variables for FAMILY_NAME
# creating a copy of dataframe for safety measures
chef_m = chef_org.copy()

# creating a new column where 1 indicates that observation has a missing family name
chef_m['m_FAMILY_NAME'] = chef_m['FAMILY_NAME'].isnull().astype(int)


# Creating Email Domain Categories
# STEP 1: splitting emails
# placeholder list
placeholder_lst = []

# looping over each email address
for index, col in chef_org.iterrows():
    
    # splitting email domain at '@'
    split_email = chef_m.loc[index, 'EMAIL'].split(sep = '@')

    # appending placeholder_lst with the results
    placeholder_lst.append(split_email)
    
# converting placeholder_lst into a DataFrame
email_df = pd.DataFrame(placeholder_lst)

# STEP 2: concatenating with original DataFrame
# Creating a copy of chef for features and safety measure
chef = chef_m.copy()

# renaming column to concatenate
email_df.columns = ['name' , 'EMAIL_DOMAIN'] 

# concatenating personal_email_domain with chef DataFrame
chef = pd.concat([chef, email_df.loc[:, 'EMAIL_DOMAIN']], 
                   axis = 1)

# printing value counts of personal_email_domain
chef.loc[: ,'EMAIL_DOMAIN'].value_counts()

# email domain types
professional_email_domains = [
    '@mmm.com',         '@amex.com',
    '@apple.com',       '@boeing.com',
    '@caterpillar.com', '@chevron.com',
    '@cisco.com',       '@cocacola.com',
    '@disney.com',      '@dupont.com',
    '@exxon.com',       '@ge.org',
    '@goldmansacs.com', '@homedepot.com',
    '@ibm.com',         '@intel.com',
    '@jnj.com',         '@jpmorgan.com',
    '@mcdonalds.com',   '@merck.com',
    '@microsoft.com',   '@nike.com',
    '@pfizer.com',      '@pg.com',
    '@travelers.com',   '@unitedtech.com',
    '@unitedhealth.com','@verizon.com',
    '@visa.com',        '@walmart.com'
    ]

personal_email_domains = [
    '@gmail.com',       '@yahoo.com',    
    '@protonmail.com'
    ]

junk_email_domains = [
    '@me.com',          '@aol.com',
    '@hotmail.com',     '@live.com', 
    '@msn.com',         '@passport.com'
    ]

# placeholder list
placeholder_lst = []  

# looping to group observations by domain type
for domain in chef['EMAIL_DOMAIN']:
        if "@" + domain in professional_email_domains:
            placeholder_lst.append('professional')
            
            
        elif "@" + domain in personal_email_domains:
            placeholder_lst.append('personal')
            
            
        elif "@" + domain in junk_email_domains:
            placeholder_lst.append('junk')
            
        else:
            print('Unknown')

            
# concatenating with original DataFrame
chef['email_domain_group'] = pd.Series(placeholder_lst)

# Step 3: One-Hot encoding
one_hot_email_domain = pd.get_dummies(chef['email_domain_group'])

# dropping orginal columns to keep only encoded ones
chef_1               = chef.drop(['email_domain_group','EMAIL','EMAIL_DOMAIN'], axis = 1)

# joining encoded columns to dataset
chef_1               = chef_1.join(one_hot_email_domain)


# Establishing outliers thresholds for analysis
# Continous
avg_time_per_site_visit_hi = 200
avg_prep_vid_time_hi       = 250
followed_rec_hi            = 80
largest_order_size_hi      = 5
avg_clicks_per_visit_lo    = 10

# Counts:
total_meals_ordered_hi            = 320
unique_meals_purchased_hi         = 10
contacts_with_customer_service_hi = 13
cancellations_before_noon_hi      = 8
late_deliveries_hi                = 17
total_photos_viewed_hi            = 800

# Target Variable
revenue_hi  =  5500

# Creating Dictionary to link variables with outlier thresholds
lst_thresholds_hi = {
    'AVG_TIME_PER_SITE_VISIT'      : avg_time_per_site_visit_hi,
    'AVG_PREP_VID_TIME'            : avg_prep_vid_time_hi,
    'TOTAL_MEALS_ORDERED'          : total_meals_ordered_hi,
    'UNIQUE_MEALS_PURCH'           : unique_meals_purchased_hi,
    'CONTACTS_W_CUSTOMER_SERVICE'  : contacts_with_customer_service_hi,
    'CANCELLATIONS_BEFORE_NOON'    : cancellations_before_noon_hi,
    'LATE_DELIVERIES'              : late_deliveries_hi,
    'TOTAL_PHOTOS_VIEWED'          : total_photos_viewed_hi,
    'REVENUE'                      : revenue_hi,
    'FOLLOWED_RECOMMENDATIONS_PCT' : followed_rec_hi,
    'LARGEST_ORDER_SIZE'           : largest_order_size_hi
    }

lst_thresholds_lo = {
    'AVG_CLICKS_PER_VISIT'         : avg_clicks_per_visit_lo
    }

# Looping over variables to create outlier flags:
for key in lst_thresholds_hi.keys():
    outlier_flag_hi(key,lst_thresholds_hi[key],chef_1)
    
for key in lst_thresholds_lo.keys():
    outlier_flag_lo(key,lst_thresholds_lo[key],chef_1)

    
# Establishing outliers thresholds for analysis
# data scatters above this point
AVG_TIME_PER_SITE_VISIT_change_hi      = 200
FOLLOWED_RECOMMENDATIONS_PCT_change_hi = 70
AVG_PREP_VID_TIME_change_lo            = 150
AVG_PREP_VID_TIME_change_hi            = 210
TOTAL_MEALS_ORDERED_change_hi          = 100

# trend changes
LARGEST_ORDER_SIZE_change_hi          = 5
AVG_CLICKS_PER_VISIT_change_lo        = 8
AVG_CLICKS_PER_VISIT_change_hi        = 16
UNIQUE_MEALS_PURCH_change_hi          = 9
CONTACTS_W_CUSTOMER_SERVICE_change_hi = 10
PRODUCT_CATEGORIES_VIEWED_change_hi   = 5
CANCELLATIONS_BEFORE_NOON_change_hi   = 8
MOBILE_LOGINS_change_hi               = 1
PC_LOGINS_change_hi                   = 6
WEEKLY_PLAN_change_hi                 = 15
LATE_DELIVERIES_change_lo             = 8
TOTAL_MEALS_ORDERED_change_hi         = 100


# Different at __ 
AVG_TIME_PER_SITE_VISIT_change_at   = 50
MEDIAN_MEAL_RATING_change_at1       = 3
MEDIAN_MEAL_RATING_change_at2       = 4
MEDIAN_MEAL_RATING_change_at3       = 5
TOTAL_MEALS_ORDERED_change_at       = 15
UNIQUE_MEALS_PURCH_change_at        = 1
CANCELLATIONS_AFTER_NOON_change_at  = 0
MASTER_CLASSES_ATTENDED_change_at1  = 1
MASTER_CLASSES_ATTENDED_change_at2  = 2


# Zero Inflated
TOTAL_PHOTOS_VIEWED_change_at = 0 

# Creating Dictionary to link variables with trend thresholds
high_thresholds = {
    'AVG_TIME_PER_SITE_VISIT'        : [AVG_TIME_PER_SITE_VISIT_change_hi],
    'FOLLOWED_RECOMMENDATIONS_PCT'   : [FOLLOWED_RECOMMENDATIONS_PCT_change_hi],
    'AVG_PREP_VID_TIME'              : [AVG_PREP_VID_TIME_change_hi,
                                        AVG_PREP_VID_TIME_change_lo],
    'TOTAL_MEALS_ORDERED'            : [TOTAL_MEALS_ORDERED_change_hi],
    'LARGEST_ORDER_SIZE'             : [LARGEST_ORDER_SIZE_change_hi],
    'AVG_CLICKS_PER_VISIT'           : [AVG_CLICKS_PER_VISIT_change_lo, 
                                        AVG_CLICKS_PER_VISIT_change_hi],
    'UNIQUE_MEALS_PURCH'             : [UNIQUE_MEALS_PURCH_change_hi],
    'CONTACTS_W_CUSTOMER_SERVICE'    : [CONTACTS_W_CUSTOMER_SERVICE_change_hi],
    'PRODUCT_CATEGORIES_VIEWED'      : [PRODUCT_CATEGORIES_VIEWED_change_hi],
    'CANCELLATIONS_BEFORE_NOON'      : [CANCELLATIONS_BEFORE_NOON_change_hi],
    'MOBILE_LOGINS'                  : [MOBILE_LOGINS_change_hi],
    'PC_LOGINS'                      : [PC_LOGINS_change_hi],
    'WEEKLY_PLAN'                    : [WEEKLY_PLAN_change_hi],
    'LATE_DELIVERIES'                : [LATE_DELIVERIES_change_lo],
    'TOTAL_MEALS_ORDERED'            : [TOTAL_MEALS_ORDERED_change_hi]
    }
                     
                     
at_thresholds = {
    'AVG_TIME_PER_SITE_VISIT'      : [AVG_TIME_PER_SITE_VISIT_change_at],
    'MEDIAN_MEAL_RATING'           : [MEDIAN_MEAL_RATING_change_at1, 
                                       MEDIAN_MEAL_RATING_change_at2, 
                                       MEDIAN_MEAL_RATING_change_at3],
    'UNIQUE_MEALS_PURCH'           : [UNIQUE_MEALS_PURCH_change_at],
    'CANCELLATIONS_BEFORE_NOON'    : [CANCELLATIONS_AFTER_NOON_change_at],
    'MASTER_CLASSES_ATTENDED'      : [MASTER_CLASSES_ATTENDED_change_at1,
                                       MASTER_CLASSES_ATTENDED_change_at2],
    'TOTAL_PHOTOS_VIEWED'          : [TOTAL_PHOTOS_VIEWED_change_at]
    }

# Looping over dataset to create trend based columns
for key, value in high_thresholds.items():
    i = 0
    for val in value:
        higher_change_flag(key, i, val, chef_1)
        i += 1

for key, value in at_thresholds.items():
    i = 0
    for val in value:
        at_flag(key, i, val, chef_1)
        i += 1    
    
# Dropping variables after feature engineering
clean_chef = chef_1.drop([
                          'CROSS_SELL_SUCCESS',
                          'NAME',
                          'FIRST_NAME',
                          'FAMILY_NAME',
                          'm_FAMILY_NAME'
                          ], axis = 1)    


#################################################
#############    Transformations      ###########
#################################################

# Log Transformation
# creating a copy for safety measures
clean_chef = clean_chef.copy()

#transforming only REVENUE column
clean_chef['rev_log'] = clean_chef['REVENUE'].transform(np.log)


# Standardization
# creating a copy for safety measures
clean_chef = clean_chef.copy()

# dropping revenue from standardization
chef_x      = clean_chef.drop(['REVENUE','rev_log'], axis = 1)
#clean_chef.columns
chef_target = clean_chef.loc[:,['REVENUE','rev_log']]

# standardizing with user defined function and joining target revenue
chef_std   = standard(chef_x).join(chef_target)

# Outliers
# Creating a copy of dataset for safety measure
out_chef = clean_chef.copy()

# Subsetting outliers from our linear model features by using flags
out_chef = out_chef.loc[:][out_chef['out_TOTAL_MEALS_ORDERED'] == 0]


# Defining a dictionary with variables names
variables_dict = {
    "ARD Features" : [
        'REVENUE',                     'TOTAL_MEALS_ORDERED',  
        'CONTACTS_W_CUSTOMER_SERVICE', 'AVG_PREP_VID_TIME',
        'LARGEST_ORDER_SIZE',          'MEDIAN_MEAL_RATING', 
        'out_TOTAL_MEALS_ORDERED',     'out_CONTACTS_W_CUSTOMER_SERVICE', 
        'change_AVG_PREP_VID_TIME_1',  'change_UNIQUE_MEALS_PURCH_0', 
        'change_CONTACTS_W_CUSTOMER_SERVICE_0',  
        'change_MEDIAN_MEAL_RATING_0',         'change_MEDIAN_MEAL_RATING_1',
        'change_MASTER_CLASSES_ATTENDED_0',    'change_MASTER_CLASSES_ATTENDED_1', 
        'change_TOTAL_PHOTOS_VIEWED_0',        'rev_log'
        ]
      }


################################################################################
# Train/Test Split
################################################################################

# Creating a variable for random seed
seed = 222

# Preparing response variable
out_chef_target = out_chef['rev_log']

# Preparing x-variables
out_chef_x = out_chef[variables_dict['ARD Features']].drop(['REVENUE','rev_log'], axis = 1)

# Running train/test split
X_train_out, X_test_out, y_train_out, y_test_out = train_test_split(
            out_chef_x,
            out_chef_target,
            test_size = 0.25,
            random_state = seed)


################################################################################
# Final Model (instantiate, fit, and predict)
################################################################################

# INSTANTIATING a model object
out_lr = LinearRegression()

# FITTING to the training data
out_lr_fit = out_lr.fit(X_train_out, y_train_out) 

# PREDICTING on new data
out_lr_pred = out_lr_fit.predict(X_test_out) 


################################################################################
# Final Model Score (score)
################################################################################

test_score = out_lr.score(X_test_out,  y_test_out).round(3)
test_score